In [196]:
from __future__ import print_function
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [197]:
MAX_FEATURES = 1000
GOOD_MATCH_PERCENT = 0.1

In [198]:
def display(img, cmap = None):
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap = cmap)

In [199]:
def align_images(im1, im2):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
  # Match features.
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1,descriptors2)
    matches = sorted(matches, key = lambda x:x.distance)
  # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)
  # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imwrite("matches.jpg", imMatches)
  # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32) 
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
  # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    return im1Reg, h

In [200]:
if __name__ == '__main__':
    
  # Read reference image
    refFilename = "./pngs/template2.png"
    print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
    
  # Read image to be aligned
    imFilename = "./pngs/175gLCL.png"
    print("Reading image to align : ", imFilename)
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
    print("Aligning images ...")

  # Registered image will be resotred in imReg.
  # The estimated homography will be stored in h.
    imReg, h = align_images(im, imReference)

  # Write aligned image to disk.
    outFilename = "./pngs/aligned175.png"
    print("Saving aligned image : ", outFilename)
    cv2.imwrite(outFilename, imReg)
    
  # Print estimated homography
    print("Estimated homography : \n",  h)


Reading reference image :  ./pngs/template2.png
Reading image to align :  ./pngs/175gLCL.png
Aligning images ...
Saving aligned image :  ./pngs/aligned175.png
Estimated homography : 
 [[ 4.09118088e-01  1.24643766e+00 -1.55507386e+03]
 [-1.06562670e+00  2.94978898e-02  2.45813045e+03]
 [ 4.57766751e-04  6.09158881e-05  1.00000000e+00]]
